In [1]:
import os, sys
from time import time
from glob import glob
import datetime as dt
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
from osgeo import osr, gdal

from IPython.display import clear_output

# import cupy as cp

In [ ]:
#12312312313wqeqe12312

In [ ]:
#123123123123123123

In [2]:
%%capture install_status
pip install netCDF4 #rasterio

In [3]:
import netCDF4 as nc

In [4]:
# 所有mask文件
mask_list = glob('/content/drive/MyDrive/2024/ai_pepper/era5/county_mask1/*.tif')
mask_list.sort()
print(len(mask_list))

99


In [5]:
# 遍历所有 mask 文件，获取某个county的这个变量的数据
mask_dict = {}
for mask_file in mask_list:
    fname = os.path.basename(mask_file)
    location = fname.split('.')[0]
    mask_dict[location] = {}
    # read mask tiff
    # print('opening mask file')
    gd = gdal.Open(mask_file)
    garr = gd.GetRasterBand(1).ReadAsArray()
    # garrc = cp.asarray(garr)
    # garr = cp.where(garrc < -9000, cp.nan, garrc)

    mask_dict[location] = garr

In [7]:
geotrans = [-125.05, 0.1, 0, 50.05, 0, -0.1]
vlist2 = ['evabs', 'evavt', 'sp', 'sshf', 'ssrd', 'strd', 'tp']
vlist1 = ['t2m', 'rsn', 'sde', 'stl1', 'stl2', 'stl3', 'stl4', 'tsn', 'swvl1', 'swvl2', 'swvl3', 'swvl4']

for year in range(2020, 2021):
    # 获取某一年的文件
    flist = glob(f'/content/drive/MyDrive/2024/ai_pepper/era5/unzip/{year}/*')
    flist.sort()

    # 遍历该年的所有netcdf文件， 确定是part1还是part2
    for ncf in flist:
        ncfname = os.path.basename(ncf)
        yearmonth = ncfname.split('_')[0]
        month_int = int(yearmonth.split('-')[1])

        # 跳过已经完成的部分
        if year == 2020:
            if month_int not in [9, 10]:
                continue
        # 所有county数据保存在 2 个大表里
        # 每个大表都有一个 time 列表，然后每个 county 单独一个 dict
        # 每个 county 下面每个变量是一个表
        # all_dict1 = {'time': []}
        # all_dict2 = {'time': []}
        outd = {'time': []}
        # read basic variables
        with nc.Dataset(ncf,'r') as ds:
            var_all = list(ds.variables.keys())
            file_type = 0
            if ncf.split('_')[-1] == 'part1.nc':
                file_type = 1
            elif ncf.split('_')[-1] == 'part2.nc':
                file_type = 2
            else:
                raise ValueError('Unknown file type')
            if file_type == 1:
                for v in vlist1:
                    if v not in var_all:
                        raise ValueError(f'Part1 file. Variable not complete. Missing: {v}')
            elif file_type == 2:
                for v in vlist2:
                    if v not in var_all:
                        raise ValueError(f'Part2 file. Variable not complete. Missing: {v}')
            else:
                raise ValueError('Error during variable completeness check')

        # set variable list
        if file_type == 1:
            vlist = vlist1
            # continue
        elif file_type == 2:
            vlist = vlist2

        # 打开netcdf文件，先获取时间信息
        with nc.Dataset(ncf,'r') as ds:
            timevar = ds.variables['time'][:]
            timelist = [
                dt.datetime(1900, 1, 1, 0, 0) + dt.timedelta(hours = int(x)) for x in timevar
                ]

            # 根据 file_type 选择添加数据到 all_dict1 还是 all_dict2
            if file_type == 1:
                vlist = vlist1
            else:
                vlist = vlist2
            outd['time'].extend(timelist)
            # 遍历所有变量名，获取全美的数据
            for v in vlist:
                with open(f'/content/drive/MyDrive/2024/ai_pepper/era5/mask2_log_YuhuaSitu2.log','a') as fp:
                    now_str = dt.datetime.strftime(dt.datetime.now(), '%Y%m%dT%H%M%S')
                    fp.writelines(f'{ncf} - {v} - {now_str}\n')
                arr = ds.variables[v][:]   # [time, lat, lon]
                # arr2 = cp.asarray(arr1.data)
                # arr = cp.where(arr2 < -30000, cp.nan, arr2)

                for location in mask_dict.keys():
                    print(ncf)
                    print(v)
                    print(location)
                    if location not in outd.keys():
                        outd[location] = {}
                    if v not in outd[location].keys():
                        outd[location][v] = np.array([])
                    garr1 = mask_dict[location]
                    garr_inds = np.where(garr1 > -10)
                    # arr_masked = garr1 * arr
                    arr_masked = arr[:, garr_inds[0], garr_inds[1]]
                    arr_out = np.mean(arr_masked, axis=1)
                    # arr_out = cp.nanmean(arr_masked, axis=(1,2))
                    # print('einsum')
                    # arr_out = np.einsum("ijk,jk->ijk", arr, garr)
                    # arr_out = np.einsum("ijk->i", arr_masked)
                    print('saving to outd (dictionary)')
                    outd[location][v] = np.concatenate(
                        [outd[location][v], arr_out],
                        axis=0
                        )
                    clear_output(wait=True)


        # 每个月保存一次
        partname = ncfname.split('_')[1].split('.')[0]
        errlist = []
        for location in outd.keys():
            if location == 'time':
                continue
            os.makedirs(f'/content/drive/MyDrive/2024/ai_pepper/era5/county_env2/{location}', exist_ok=True)
            try:
                outd1 = {}
                for key in outd[location].keys():
                    outd1[key] = outd[location][key]# .get()
                t2 = pd.DataFrame(outd1)
                t2['datetime'] = outd['time']
                t2.to_csv(f'/content/drive/MyDrive/2024/ai_pepper/era5/county_env2/{location}/{location}_{yearmonth}_{partname}.csv', index=False)
            except:
                print(f'error with location: {location}')
                errlist.append(location)


/content/drive/MyDrive/2024/ai_pepper/era5/unzip/2020/2020-10_part2.nc
tp
Iowa_Wright
saving to outd (dictionary)
